To run in Colab (skip otherwise)

In [ ]:
!curl -sS https://apertium.projectjj.com/apt/install-release.sh | sudo bash
!apt install apertium-all-dev lexd

Make sure that `hfst-guess bezhta.guesser.hfst` is in the same directory

# Coverage

## Unrecognized tokens

In [ ]:
def guess_word(word):
    output = os.popen(f"echo {word} | hfst-guess bezhta.guesser.hfst -n 100").read().rstrip('\n\n')
    parses = []
    for el in output.split('\n'):
        parses.append(':'.join(el.split('\t')))

    return parses

In [ ]:
def check_guesser_coverage(path):
    with open(path, 'r', encoding='utf-8') as file:
        file = file.read()
        words = re.findall(r"(?P<num>\d+) \^(?P<word>[а-яōāēӣӯӏᴴ]+)\/.*\$", file)
        wd = {}
        for word in words:
            guess = guess_word(word[1])
            if guess == ['']:
                guess = []
            wd[word[1]] = {'number': int(word[0]), 'guess':guess, 'len_guess':len(guess)}

    n_recog = 0
    n_unrecog = 0
    for word in wd.keys():
        if wd[word]['len_guess']:
            n_recog += wd[word]['number']
        else:
            n_unrecog += wd[word]['number']

    print('recog: ', n_recog)
    print('unrecog: ', n_unrecog)
    print('coverage: ', n_recog/(n_recog+n_unrecog))

    return wd

In [ ]:
gospel = check_guesser_coverage('unrecog-gospel.txt')

recog:  4019
unrecog:  995
coverage:  0.8015556441962505


In [ ]:
gospel = check_guesser_coverage('unrecog-gospel.txt')

recog:  4019
unrecog:  995
coverage:  0.8015556441962505


In [ ]:
gospel = check_guesser_coverage('unrecog-gospel.txt')

recog:  4019
unrecog:  995
coverage:  0.8015556441962505


In [ ]:
gospel = check_guesser_coverage('unrecog-gospel.txt')

recog:  4019
unrecog:  995
coverage:  0.8015556441962505


## Full corpora

In [ ]:
def check_guesser_coverage_full(path):
   with open(path, 'r', encoding='utf-8') as file:
    un = []
    total = 0.0
    tokens_recognised = 0.0
    unique = 0.0
    lines = file.readlines()
    for l in lines:
        parses = [guess_word(i.strip()) for i in l.split()]
        for w in parses:
          if w[0] != '':
            tokens_recognised += 1
          total +=1
    print('tokens: ', total)
    print('tokens recognised: ', tokens_recognised)
    print('tokens unrecognised: ', total - tokens_recognised)
    print('token coverage: ', tokens_recognised / total)

In [ ]:
luke_full = check_guesser_coverage_full('text-Luke.txt')

tokens:  19739.0
tokens recognised:  16011.0
tokens unrecognised:  3728.0
token coverage:  0.8111353158721313


In [ ]:
prov_full = check_guesser_coverage_full('text-Prov.txt')

tokens:  11644.0
tokens recognised:  9192.0
tokens unrecognised:  2452.0
token coverage:  0.7894194434902095


In [ ]:
text_1_full = check_guesser_coverage_full('text-turkey.txt')

tokens:  86.0
tokens recognised:  71.0
tokens unrecognised:  15.0
token coverage:  0.8255813953488372


In [ ]:
text_2_full = check_guesser_coverage_full('text-life.txt')

tokens:  116.0
tokens recognised:  90.0
tokens unrecognised:  26.0
token coverage:  0.7758620689655172


# Accuracy

## All tokens

In [ ]:
def guess_word_acc(word):
    output = os.popen(f"echo {word} | hfst-guess bezhta.guesser.hfst -n 100").read().rstrip('\n\n')
    parses = []
    for el in output.split('\n'):
      if output.split('\n')[0] != '':
        parse = el.split('\t')[1]
        parses.append(re.sub('\[GUESS_CATEGORY=\w+\]', '', parse))
    if parses == ['']:
        parses = []
    return parses

In [ ]:
def check_guesser_acc(path):
  with open(path, 'r', encoding='utf-8') as file:
      file = file.read()
      fully_guessed = 0.0
      recog = 0.0
      total = 0.0
      tags_guessed = 0.0
      seen = []
      words = re.findall(r"\^(?P<word>[а-яōāēӣӯӏᴴ]+)\/(?P<parse>.*)\$", file)
      for w in words:
        if w not in seen:
          guesses = guess_word_acc(w[0])
          if len(guesses) != 0:
            if w[1] in guesses:
                fully_guessed += 1
            #else:
                #print('FAIL')
                #print('standard:', w[0], w[1])
                #print(guesses)
            guessed_tags = [re.findall(r"<.*>", l)[0] for l in guesses]
            true_tags = re.findall(r"<.*>", w[1])[0]
            if true_tags in guessed_tags:
              tags_guessed += 1
            recog += 1
          total +=1
          seen.append(w)
      print('total:', total)
      print('recognised:', recog)
      print('types recognised:', recog/total)
      print('fully_guessed:', fully_guessed)
      print('fully_guessed_all:', fully_guessed / total)
      print('fully_guessed_recog:', fully_guessed / recog)
      print('tags_guessed:', tags_guessed / total)

In [ ]:
check_guesser_acc('text-1-gold.txt')

total: 76.0
recognised: 64.0
types recognised: 0.8421052631578947
fully_guessed: 26.0
fully_guessed_all: 0.34210526315789475
fully_guessed_recog: 0.40625
tags_guessed: 0.34210526315789475


In [ ]:
check_guesser_acc('text-2-gold.txt')

total: 79.0
recognised: 68.0
types recognised: 0.8607594936708861
fully_guessed: 23.0
fully_guessed_all: 0.2911392405063291
fully_guessed_recog: 0.3382352941176471
tags_guessed: 0.2911392405063291


## Perfect tokens

In [ ]:
def check_guesser_acc_perfect(path):
  with open(path, 'r', encoding='utf-8') as file:
      file = file.read()
      fully_guessed = 0.0
      recog = 0.0
      total = 0.0
      tags_guessed = 0.0
      words = re.findall(r"\^(?P<word>[а-яōāēӣӯӏᴴ]+)\/(.*<(n|v|num|dem)>.*)\$", file)
      seen = []
      for w in words:
        if w not in seen:
          guesses = guess_word_acc(w[0])
          if len(guesses) != 0:
            if w[1] in guesses:
                fully_guessed += 1
            #else:
               #print('FAIL')
               #print('standard:', w[0], w[1])
               #print(guesses)
            guessed_tags = [re.findall(r"<.*>", l)[0] for l in guesses]
            true_tags = re.findall(r"<.*>", w[1])[0]
            if true_tags in guessed_tags:
              tags_guessed += 1
            recog += 1
          total +=1
          seen.append(w)

      print('total:', total)
      print('recognised:', recog)
      print('types recognised:', recog/total)
      print('fully_guessed:', fully_guessed)
      print('fully_guessed_all:', fully_guessed / total)
      print('fully_guessed_recog:', fully_guessed / recog)
      print('tags_guessed:', tags_guessed / total)

In [ ]:
check_guesser_acc_perfect('text-1-gold.txt')

total: 52.0
recognised: 46.0
types recognised: 0.8846153846153846
fully_guessed: 26.0
fully_guessed_all: 0.5
fully_guessed_recog: 0.5652173913043478
tags_guessed: 0.5


In [ ]:
check_guesser_acc_perfect('text-2-gold.txt')

total: 51.0
recognised: 46.0
types recognised: 0.9019607843137255
fully_guessed: 23.0
fully_guessed_all: 0.45098039215686275
fully_guessed_recog: 0.5
tags_guessed: 0.45098039215686275


In [ ]:
check_guesser_acc_perfect('text-2-gold.txt')

total: 51.0
recognised: 46.0
types recognised: 0.9019607843137255
fully_guessed: 23.0
fully_guessed_all: 0.45098039215686275
fully_guessed_recog: 0.5
tags_guessed: 0.45098039215686275


## Verbs & nouns separately

In [ ]:
def check_guesser_acc_nouns(path):
  with open(path, 'r', encoding='utf-8') as file:
      file = file.read()
      fully_guessed = 0.0
      recog = 0.0
      total = 0.0
      tags_guessed = 0.0
      words = re.findall(r"\^(?P<word>[а-яōāēӣӯӏᴴ]+)\/(.*<(n)>.*)\$", file)
      seen = []
      for w in words:
        if w not in seen:
          print(w)
          guesses = guess_word_acc(w[0])
          if len(guesses) != 0:
            if w[1] in guesses:
                fully_guessed += 1
            #else:
               #print('FAIL')
               #print('standard:', w[0], w[1])
               #print(guesses)
            guessed_tags = [re.findall(r"<.*>", l)[0] for l in guesses]
            true_tags = re.findall(r"<.*>", w[1])[0]
            if true_tags in guessed_tags:
              tags_guessed += 1
            recog += 1
          total +=1
          seen.append(w)

      print('total:', total)
      print('recognised:', recog)
      print('types recognised:', recog/total)
      print('fully_guessed:', fully_guessed)
      print('fully_guessed_all:', fully_guessed / total)
      print('fully_guessed_recog:', fully_guessed / recog)
      print('tags_guessed:', tags_guessed / total)

In [ ]:
check_guesser_acc_nouns('text-1-gold.txt')

('базаййагъой', 'базай<n><obl><cum><ess>', 'n')
('бикълабашейоьлъи', 'бикълабашейоьлъи<n>', 'n')
('лӏи', 'лӏи<n><abs>', 'n')
('аьлӏаьъаьш', 'аьлӏ<n><obl><in><ess><abl>', 'n')
('миц', 'миц<n><abs>', 'n')
('бикълабашейолъилӏона', 'бикълабашейолъи<n><abs><quot><add>', 'n')
('ъаьᴴгъаь', 'ъаьᴴгъаь<n><abs>', 'n')
('бикълабашейолъина', 'бикълабашейолъи<n><abs><add>', 'n')
('гьекьар', 'гьекьар<n><abs>', 'n')
('оьлоьхъаьн', 'оьлоьхъаьн<n><abs>', 'n')
('кӏималид', 'кӏима<n><obl><ins>', 'n')
('кӏима', 'кӏима<n><abs>', 'n')
('цӏитӏад', 'цӏитӏ<n><obl><ins>', 'n')
('роьъил', 'роьъил<n><abs>', 'n')
('инкар', 'инкар<n><abs>', 'n')
('къимат', 'къимат<n><abs>', 'n')
('гемо', 'гемо<n><abs>', 'n')
('гьикматна', 'гьикмат<n><abs><add>', 'n')
total: 18.0
recognised: 12.0
types recognised: 0.6666666666666666
fully_guessed: 3.0
fully_guessed_all: 0.16666666666666666
fully_guessed_recog: 0.25
tags_guessed: 0.16666666666666666


In [ ]:
check_guesser_acc_nouns('text-2-gold.txt')

('дунналнакъодā', 'дуннал<n><abs><add><irr>', 'n')
('аллагь', 'аллагь<n><abs>', 'n')
('интернетна', 'интернет<n><abs><conj>', 'n')
('кӏетӏослъина', 'кӏетӏослъи<n><abs><add>', 'n')
('телевизорлана', 'телевизор<n><obl><erg><add>', 'n')
('ийо', 'ийо<n><abs>', 'n')
('аболкъодā', 'або<n><obl><dat><irr>', 'n')
('руслан', 'руслан<n><abs>', 'n')
('оьмроь', 'оьмроь<n><abs>', 'n')
('эркенлъина', 'эркенлъи<n><abs><add>', 'n')
('або', 'або<n><abs>', 'n')
('загьматлъи', 'загьмат<n><abs>', 'n')
('хунзахъ', 'хунзахъ<n><abs>', 'n')
('жо', 'жо<n><abs>', 'n')
('оьмроьлаькьаь', 'оьмроь<n><obl><sup><ess>', 'n')
('оьмроьнаь', 'оьмроь<n><abs><add>', 'n')
('абона', 'або<n><abs><add>', 'n')
('аьгаьрлъина', 'аьгаьрлъи<n><abs><add>', 'n')
('заманна', 'заман<n><abs><add>', 'n')
('телевизорлиъ', 'телевизор<n><obl><in><ess>', 'n')
('интернетлиъ', 'интернет<n><obl><in><ess>', 'n')
('аьгаьрлъли', 'аьгаьрлъли<n><abs>', 'n')
total: 22.0
recognised: 19.0
types recognised: 0.8636363636363636
fully_guessed: 3.0
fully_gue

In [ ]:
def check_guesser_acc_verbs(path):
  with open(path, 'r', encoding='utf-8') as file:
      file = file.read()
      fully_guessed = 0.0
      recog = 0.0
      total = 0.0
      tags_guessed = 0.0
      words = re.findall(r"\^(?P<word>[а-яōāēӣӯӏᴴ]+)\/(.*<(v)>.*)\$", file)
      seen = []
      for w in words:
        if w not in seen:
          guesses = guess_word_acc(w[0])
          if len(guesses) != 0:
            if w[1] in guesses:
                fully_guessed += 1
            else:
               print('FAIL')
               print('standard:', w[0], w[1])
               print(guesses)
            guessed_tags = [re.findall(r"<.*>", l)[0] for l in guesses]
            true_tags = re.findall(r"<.*>", w[1])[0]
            if true_tags in guessed_tags:
              tags_guessed += 1
            recog += 1
          total +=1
          seen.append(w)

      print('total:', total)
      print('recognised:', recog)
      print('types recognised:', recog/total)
      print('fully_guessed:', fully_guessed)
      print('fully_guessed_all:', fully_guessed / total)
      print('fully_guessed_recog:', fully_guessed / recog)
      print('tags_guessed:', tags_guessed / total)

In [ ]:
check_guesser_acc_verbs('text-1-gold.txt')

FAIL
standard: йуьчӏйаьгъечӏе <IV>уьчӏ<IV>аьгъ<v><neg><pfv.cvb>
['йуьчӏ<antip><pl>гъо<v><neg><pfv.cvb>', 'йуьчӏ<antip><pl>гъ<v><neg><pfv.cvb>', 'йуьчӏйаьгъо<v><neg><pfv.cvb>', 'йуьчӏйаьгъе<v><neg><pfv.cvb>', 'йуьчӏйаьгъи<v><neg><pfv.cvb>', 'йуьчӏйаьгъоь<v><neg><pfv.cvb>', 'йуьчӏйаьгъа<v><neg><pfv.cvb>']
FAIL
standard: богьлъол <III>ов<v><ant.cvb>
['б<m>гь<v><ant.cvb>', 'бог<v><ant.cvb>', 'богь<n><obl><dat>', 'богьлъо<n><obl><dat>', 'богьл<n><in><ess><lat>', 'богьл<n><obl><in><ess><lat>', 'богьлъ<n><obl><dat>', 'богьлхъ<n><obl><dat>', 'богьлъ<n><dat>', 'богьлъ<obl><dat>', 'богьлъоа<n><obl><dat>', 'богьлъое<n><obl><dat>', 'богьлъонзил<n><obl><dat>', 'богьлъоам<n><obl><dat>', 'богьлъоо<n><obl><dat>', 'богьлъохāь<n><obl><dat>', 'богьлъоᴴ<n><obl><dat>']
FAIL
standard: йуьчӏна <II>уьчӏ<v><pfv.cvb>
['йуьчӏна<n><pl><abs>', 'йуьчӏ<v><pfv.cvb>', 'йуьчӏни<n><pl><abs>', 'йуьч<num><dstr><abs>', 'йуьча<num><dstr><abs>', 'йуьчӏ<num><part>', 'йуьчӏн<n><pl><abs>', 'йуьчӏнаь<n><pl><abs>', 'йуьчӏна<v><im

In [ ]:
check_guesser_acc_verbs('text-2-gold.txt')

FAIL
standard: йовал <IV>ов<v><inf>
['йо<v><antip><inf>', 'йо<v><pl><inf>', 'йо<v><pst.ptsp><obl><dat>', 'йо<n><pl><obl><dat>', 'йо<n><obl><dat>', 'й<m>в<v><inf>', 'йова<n><pl><obl><dat>', 'йова<n><obl><dat>', 'йо<antip><v><inf>', 'йо<pl><v><inf>', 'йоᴴ<n><pl><obl><dat>', 'йов<n><pl><obl><dat>', 'йови<n><pl><obl><dat>', 'йоваь<n><pl><obl><dat>', 'йово<n><obl><dat>', 'йова<v><inf>', 'йове<v><inf>', 'йови<v><inf>', 'йово<v><inf>', 'йове<n><pl><obl><dat>', 'йово<n><pl><obl><dat>', 'йовоь<n><pl><obl><dat>', 'йов<dat>', 'йов<n><obl><dat>', 'йовче<n><obl><dat>', 'йованзил<n><obl><dat>', 'йовйо<n><obl><dat>', 'йова<n><dat>', 'йоваа<n><obl><dat>', 'йоваам<n><obl><dat>', 'йовахāь<n><obl><dat>', 'йоваᴴ<n><obl><dat>']
FAIL
standard: йегāйо <IV>егā<v><pst>
['йегā<v><pst>']
FAIL
standard: бегāкъала <III>ега<v><neg.opt>
['<III>ега<v><proh><conj>', '<hpl>ега<v><proh><conj>', 'беге<v><proh><conj>', 'беги<v><proh><conj>', 'бего<v><proh><conj>', 'бегоь<v><proh><conj>', 'бегāᴴкъе<v><imp><conj>', 'бегāᴴкъ